# Quantitative Momentum Strategy




## Library Imports


In [105]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
from datetime import datetime, timedelta

## Importing Our List of Stocks


In [107]:
# Fetch current S&P 500 tickers from Wikipedia
def fetch_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    sp500_table = pd.read_html(url)  # This reads all tables on the Wikipedia page
    sp500_df = sp500_table[0]  # The first table contains the S&P 500 data
    return sp500_df['Symbol'].tolist()  # Return list of tickers

## Obtaining current date

In [109]:
def get_dates(T):
    # Get the current date
    current_date = datetime.now().date()
    
    # Calculate the date two years before
    # If you want to account for leap years, you can use the replace method to go back exactly two years
    two_years_before = current_date.replace(year=current_date.year - T)

    return current_date, two_years_before

## Building the High Quality Momentum Strategy

To identify high-quality momentum, we select stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

In [111]:
hqm_columns = ['Ticker', 
               'Price',  
               'Number of Shares to Buy',
              'One-Year Price Return',
              'One-Year Return Percentile',
              'Six-Month Price Return',
              'Six-Month Return Percentile',
              'Three-Month Price Return',
              'Three-Month Return Percentile',
              'One-Month Price Return',
              'One-Month Return Percentile',
              'HQM Score']
hqm_dataframe= pd.DataFrame(columns = hqm_columns)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score


## Importing Financial Data for desired stocks

Here, financial data from the last 2 years is imported from using 'yfinance'.

In [113]:
import yfinance as yf
# Assume 'stocks' is a DataFrame or Series containing tickers
# Assume 'hqm_columns' is defined with the necessary column names
results = []  # Initialize a list to collect new rows

# Fetch S&P 500 tickers
tickers = fetch_sp500_tickers()

T = 2 # 2 years of financial data desired

#Calls function to get current date, and the date T years ago.
current_date, two_years_before = get_dates(2)

# Loop through the tickers
for stock in tickers:
    try:
        # Download historical data for the stock
        data = yf.download(stock, start=two_years_before, end=current_date)

        # Check if 'Close' column exists and has data
        if 'Close' in data.columns and not data['Close'].empty:
            close = data['Close'].iloc[-1]  # Most recent closing price

            # Calculate actual dates
            last_date = data.index[-1]  # Most recent date in the data
            date_1yr_ago = last_date - pd.DateOffset(years=1)
            date_6m_ago = last_date - pd.DateOffset(months=6)
            date_3m_ago = last_date - pd.DateOffset(months=3)
            date_1m_ago = last_date - pd.DateOffset(months=1)

            # Find the closest available dates using loc
            closest_1yr = data.loc[:date_1yr_ago]['Close'].iloc[-1] if not data.loc[:date_1yr_ago].empty else float('nan')
            closest_6m = data.loc[:date_6m_ago]['Close'].iloc[-1] if not data.loc[:date_6m_ago].empty else float('nan')
            closest_3m = data.loc[:date_3m_ago]['Close'].iloc[-1] if not data.loc[:date_3m_ago].empty else float('nan')
            closest_1m = data.loc[:date_1m_ago]['Close'].iloc[-1] if not data.loc[:date_1m_ago].empty else float('nan')

            # Calculate returns
            return1yr = (close - closest_1yr) / closest_1yr if pd.notna(closest_1yr) else float('nan')
            return6m = (close - closest_6m) / closest_6m if pd.notna(closest_6m) else float('nan')
            return3m = (close - closest_3m) / closest_3m if pd.notna(closest_3m) else float('nan')
            return1m = (close - closest_1m) / closest_1m if pd.notna(closest_1m) else float('nan')

            # Create a new row and add it to the results list
            new_row = [stock, close, 'N/A', return1yr, '', return6m, '', return3m, '', return1m, '', 'N/A']
            results.append(new_row)

    except Exception as e:
        print(f"Error processing {stock}: {e}")
        continue
        
# Create the final DataFrame from the results list
hqm_dataframe = pd.DataFrame(results, columns=hqm_columns)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [115]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

# List of return columns
return_columns = [
    'One-Year Price Return',
    'Six-Month Price Return',
    'Three-Month Price Return',
    'One-Month Price Return'
]
# Drop rows with NaN values in the specified columns
hqm_dataframe = hqm_dataframe.dropna(subset=return_columns)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,132.149994,N/A,0.477806,,0.728471,,0.340128,,0.055174,,N/A
1,AOS,80.349998,N/A,0.092157,,-0.055817,,-0.011077,,-0.024879,,N/A
2,ABT,114.400002,N/A,0.112299,,-0.046984,,0.106062,,0.027760,,N/A
3,ABBV,197.009995,N/A,0.329352,,0.112736,,0.215061,,0.040784,,N/A
4,ACN,343.989990,N/A,0.049582,,-0.103516,,0.194161,,0.064490,,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,XYL,129.089996,N/A,0.258432,,0.009699,,-0.058425,,-0.008144,,N/A
497,YUM,134.809998,N/A,0.039880,,-0.026994,,-0.047548,,-0.002737,,N/A
498,ZBRA,328.869995,N/A,0.191774,,0.127619,,0.086204,,0.020385,,N/A
499,ZBH,114.480003,N/A,-0.041607,,-0.087881,,0.009168,,0.037332,,N/A


In [116]:

for row in hqm_dataframe.index:
    for time_period in time_periods:
      
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,132.149994,N/A,0.477806,0.897384,0.728471,1.0,0.340128,0.993964,0.055174,0.631791,N/A
1,AOS,80.349998,N/A,0.092157,0.374245,-0.055817,0.241449,-0.011077,0.259557,-0.024879,0.120724,N/A
2,ABT,114.400002,N/A,0.112299,0.406439,-0.046984,0.253521,0.106062,0.629779,0.027760,0.400402,N/A
3,ABBV,197.009995,N/A,0.329352,0.758551,0.112736,0.629779,0.215061,0.937626,0.040784,0.521127,N/A
4,ACN,343.989990,N/A,0.049582,0.309859,-0.103516,0.16499,0.194161,0.907445,0.064490,0.704225,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,XYL,129.089996,N/A,0.258432,0.67002,0.009699,0.380282,-0.058425,0.173038,-0.008144,0.179074,N/A
497,YUM,134.809998,N/A,0.039880,0.289738,-0.026994,0.287726,-0.047548,0.191147,-0.002737,0.195171,N/A
498,ZBRA,328.869995,N/A,0.191774,0.567404,0.127619,0.653924,0.086204,0.547284,0.020385,0.342052,N/A
499,ZBH,114.480003,N/A,-0.041607,0.211268,-0.087881,0.197183,0.009168,0.32998,0.037332,0.468813,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [118]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles= []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row,f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)


hqm_dataframe



,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,132.149994,N/A,0.477806,0.897384,0.728471,1.0,0.340128,0.993964,0.055174,0.631791,0.880785
1,AOS,80.349998,N/A,0.092157,0.374245,-0.055817,0.241449,-0.011077,0.259557,-0.024879,0.120724,0.248994
2,ABT,114.400002,N/A,0.112299,0.406439,-0.046984,0.253521,0.106062,0.629779,0.027760,0.400402,0.422535
3,ABBV,197.009995,N/A,0.329352,0.758551,0.112736,0.629779,0.215061,0.937626,0.040784,0.521127,0.711771
4,ACN,343.989990,N/A,0.049582,0.309859,-0.103516,0.16499,0.194161,0.907445,0.064490,0.704225,0.52163
...,...,...,...,...,...,...,...,...,...,...,...,...
496,XYL,129.089996,N/A,0.258432,0.67002,0.009699,0.380282,-0.058425,0.173038,-0.008144,0.179074,0.350604
497,YUM,134.809998,N/A,0.039880,0.289738,-0.026994,0.287726,-0.047548,0.191147,-0.002737,0.195171,0.240946
498,ZBRA,328.869995,N/A,0.191774,0.567404,0.127619,0.653924,0.086204,0.547284,0.020385,0.342052,0.527666
499,ZBH,114.480003,N/A,-0.041607,0.211268,-0.087881,0.197183,0.009168,0.32998,0.037332,0.468813,0.301811


## Selecting the 50 Best Momentum Stocks

We can identify the 50 best momentum stocks by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [120]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]

hqm_dataframe.drop(labels=['index','level_0'],axis=1,inplace = True)
hqm_dataframe


KeyError: "['index', 'level_0'] not found in axis"

## Calculating the Number of Shares to Buy

Here, I create `portfolio_input` function to determine the desired size of the portfolio. Then I'll calculate the Number of shares to buy for each stock.

In [ ]:
def portfolio_input():
    while True:
        try:
            # Ask for user input
            portfolio_size = float(input("Enter your portfolio size: "))
            
            # Check if the portfolio size is positive
            if portfolio_size <= 0:
                print("Portfolio size must be a positive number. Please try again.")
            else:
                return portfolio_size
        except ValueError:
            print("Invalid input. Please enter a numeric value.")
            
portfolio_size = portfolio_input()

In [ ]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)

for i in range(len(hqm_dataframe)):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])



## Formatting Our Excel Output using xlsxwriter


In [ ]:
writer = pd.ExcelWriter('equal_momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name="Equal weighted HQM Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File


In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
    'A':[' Ticker', string_template], 
    'B':[' Price', dollar_template],  
    'C':['Number of Shares to Buy', integer_template],
    'D':['One-Year Price Return', percent_template],
    'E':['One-Year Return Percentile', percent_template],
    'F':['Six-Month Price Return', percent_template],
    'G':['Six-Month Return Percentile', percent_template],
    'H':['Three-Month Price Return', percent_template],
    'I':['Three-Month Return Percentile', percent_template],
    'J':['One-Month Price Return', percent_template],
    'K':['One-Month Return Percentile', percent_template],
    'L':['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Equal weighting High Quality Momentum Strategy'].set_column(f'{column}:{column}', 22, column_formats[column][1])



## Saving Our Excel Output


In [ ]:
writer.close()

## Weighted Momentum Strategy

Instead of equal weighting, we can adjust the strategy so that a higher momentum stocks have a larger weighting in the portfolio, proportional to their HQM score

In [ ]:

# Normalize scores to get weights
total_momentum_score_sum = hqm_dataframe['HQM Score'].sum()
hqm_dataframe['Weight'] = hqm_dataframe['HQM Score'] / total_momentum_score_sum


In [ ]:
# Calculation the postion size and hence number of shares to buy for each stock based on their HQM weighting.
for i in range(len(hqm_dataframe)):
    
    position_size = position_size = portfolio_size * hqm_dataframe.loc[i, 'Weight']

    
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])


hqm_dataframe

In [ ]:

writer = pd.ExcelWriter('unequal_momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name="Weighted HQM Strategy", index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'
string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
    'A':[' Ticker', string_template], 
    'B':[' Price', dollar_template],  
    'C':['Number of Shares to Buy', integer_template],
    'D':['One-Year Price Return', percent_template],
    'E':['One-Year Return Percentile', percent_template],
    'F':['Six-Month Price Return', percent_template],
    'G':['Six-Month Return Percentile', percent_template],
    'H':['Three-Month Price Return', percent_template],
    'I':['Three-Month Return Percentile', percent_template],
    'J':['One-Month Price Return', percent_template],
    'K':['One-Month Return Percentile', percent_template],
    'L':['HQM Score', percent_template],
    'M':['Weight', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Weighted HQM Strategy'].set_column(f'{column}:{column}', 22, column_formats[column][1])


writer.close()

In [138]:
def backtest_strategy(start_date, end_date, hqm_dataframe):
    portfolio_values = []
    benchmark_data = yf.download('^GSPC', start=start_date, end=end_date)
    benchmark_returns = benchmark_data['Close'].pct_change().dropna()

    # Convert benchmark returns to a DataFrame to align with other data
    benchmark_returns = pd.DataFrame(benchmark_returns)
    benchmark_returns.columns = ['Benchmark Return']

    for index, row in hqm_dataframe.iterrows():
        ticker = row['Ticker']
        shares = row['Number of Shares to Buy']
        
        

        stock_data = yf.download(ticker, start=start_date, end=end_date)
        stock_returns = stock_data['Close'].pct_change().dropna()

        # Align stock_returns with benchmark_returns
        combined_returns = stock_returns.to_frame().merge(benchmark_returns, left_index=True, right_index=True, how='inner')
        
        if combined_returns.empty:
            print(f"No matching data for {ticker}.")
            continue

        # Calculate portfolio value change based on stock returns and number of shares
        portfolio_value = (combined_returns['Close'] * shares).sum()
        portfolio_values.append(portfolio_value)

    # Calculate total portfolio return
    total_portfolio_return = np.sum(portfolio_values)
    total_benchmark_return = benchmark_returns['Benchmark Return'].sum()

    return total_portfolio_return, total_benchmark_return

# Define backtesting period
backtest_start_date = two_years_before
backtest_end_date = current_date

# Perform backtest
portfolio_return, benchmark_return = backtest_strategy(backtest_start_date, backtest_end_date, hqm_dataframe)
print(f"Portfolio Return: {portfolio_return:.2%}")
print(f"Benchmark Return: {benchmark_return:.2%}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('float64'), dtype('<U3')) -> None